# Renamed Dataset Verification

This notebook verifies your **newly renamed** dataset (`itc_ams_c...`).
It will plot images with their bounding boxes and labels to confirm accurate mapping.

In [ ]:
import os
import random
import yaml
import cv2
import matplotlib.pyplot as plt
from pathlib import Path

# --- Configuration ---
# Point to the local export directory where the clean data.yaml lives
PROJECT_ROOT = r"D:\I5\advance_programming\CamCurrencyProject\CurrencyFullStack\notebooks\dataset_export"
DATA_YAML = os.path.join(PROJECT_ROOT, "data.yaml")
TRAIN_IMAGES_DIR = os.path.join(PROJECT_ROOT, "train", "images")
TRAIN_LABELS_DIR = os.path.join(PROJECT_ROOT, "train", "labels")

# 1. Load Class Names from data.yaml
if os.path.exists(DATA_YAML):
    with open(DATA_YAML, 'r') as f:
        config = yaml.safe_load(f)
        class_names = config.get('names', [])
    print(f"✅ Loaded {len(class_names)} classes: {class_names}")
else:
    # Fallback if Yaml missing
    class_names = ['10000', '1000', '100', '20000', '50000', '5000', '500']
    print(f"⚠️ {DATA_YAML} not found. Using default list.")

In [ ]:
# 2. Visualize Samples
def verify_dataset(num_samples=9):
    # Find all images (jpg/png)
    all_images = list(Path(TRAIN_IMAGES).glob("*.jpg")) + \
                 list(Path(TRAIN_IMAGES).glob("*.png")) + \
                 list(Path(TRAIN_IMAGES).glob("*.jpeg"))

    if not all_images:
        print(f"❌ No images found in {TRAIN_IMAGES}")
        return
    
    print(f"Found {len(all_images)} images. Showing {num_samples} random samples...")

    # Pick random images
    samples = random.sample(all_images, min(num_samples, len(all_images)))

    plt.figure(figsize=(15, 15))
    
    for i, img_path in enumerate(samples):
        # Read Image
        img = cv2.imread(str(img_path))
        if img is None: continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape

        # Read Label
        label_path = os.path.join(TRAIN_LABELS, img_path.stem + ".txt")
        
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()
                
            for line in lines:
                parts = line.strip().split()
                cls = int(parts[0])
                # YOLO format: x_center y_center width height (normalized)
                xc, yc, bw, bh = map(float, parts[1:])
                
                # Convert to pixels
                x1 = int((xc - bw/2) * w)
                y1 = int((yc - bh/2) * h)
                x2 = int((xc + bw/2) * w)
                y2 = int((yc + bh/2) * h)
                
                # Draw
                color = (0, 255, 0)
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)
                
                label_txt = f"{class_names[cls]} ({cls})"
                cv2.putText(img, label_txt, (x1, y1-10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        else:
            print(f"⚠️ Missing label for {img_path.name}")

        # Plot
        plt.subplot(3, 3, i+1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(img_path.name, fontsize=9)

    plt.tight_layout()
    plt.show()

verify_dataset()